In [ ]:
selected_week = input("Select a Week (YYYY-MM-DD): ")

cursor = conn.cursor()

# Execute a query to count records for the previous weeks
cursor.execute(
    "SELECT q.hospital_overall_rating, l.state, q.data_date \
     FROM hospitalqualityinformation as q \
     INNER JOIN hospitallocations as l ON q.facility_id = l.hospital_fk \
     WHERE q.data_date = %s;",
    (selected_week,)
)
quality_weeks = cursor.fetchall()

In [ ]:
df = pd.DataFrame(quality_weeks, columns=["hospital_overall_rating", "state", "data_date"])
df = df.dropna()

df["hospital_overall_rating"] = pd.to_numeric(df["hospital_overall_rating"], errors='coerce')

# Calculate the average hospital_overall_rating for each state
avg_ratings = df.groupby("state")["hospital_overall_rating"].mean().sort_values(ascending=False)

# Get the top 10 and bottom 10 states
top_states = avg_ratings.head(10)
bottom_states = avg_ratings.tail(10)

# Scatter plot for the top 5 states
plt.figure(figsize=(10, 6))
plt.scatter(top_states.index, top_states.values, color='green', label='Top 10 States', s=100)
plt.scatter(bottom_states.index, bottom_states.values, color='red', label='Bottom 10 States', s=100)
plt.title(f'Average Hospital Overall Rating by State in week: {selected_week}')
plt.xlabel('State')
plt.ylabel('Average Hospital Overall Rating')
plt.legend()
plt.show()